Topic Model of Reddit & Bluesky Data

In [1]:
from bertopic import BERTopic

In [3]:
import json
    

In [12]:
# Step 1: read json file, store as list of dicts
r_posts_lst = []
with open('reddit_posts_cleaned.json', 'r') as file:
    for line in file:
        r_posts_lst.append(json.loads(line))

In [95]:
r_posts_lst

[[{'author': 'Active_Appeal_2673',
   'author_fullname': 't2_1hzvzxepdw',
   'created_utc': 1737,
   'permalink': 'https://old.reddit.com/r/AmIOverreacting/comments/1iazjin/aio_my_friend_wants_to_kick_me_out_after_i_lost/',
   'name': 't3_1iazjin',
   'subreddit_id': 't5_4uoy2u',
   'subreddit': 'AmIOverreacting',
   'num_crossposts': 6,
   'num_comments': 4983,
   'score': 59203,
   'selftext': 'Throwaway account, my main has too much personal shit on it.\n\n\nLong story short, I’ve known this dude for 15 years… he’s always been a little flakey but we’ve been friends forever… when he lost everything I gave him a place to sleep and live, I literally didn’t charge him a cent until he figured his life out and moved out. \n\nI, like many others, lost my home in the LA wildfires. You can’t even imagine how fucking terrible this has been for so many of us. I’ve literally been in tears most days… I have nothing except a car and some clothes… I’ve only been staying on his fucking couch for le

In [14]:
r_comments_lst = []
with open('reddit_comments_cleaned.json', 'r') as file:
    for line in file:
        r_comments_lst.append(json.loads(line))

In [96]:
r_text_list = []
id_list = []

for post in r_posts_lst[0]:
    title = post["title"]
    content = post["selftext"]
    author = post["author_fullname"]
    text = title + content
    count = 0
    for comment in r_comments_lst[0]:
        if comment['link_id'] == post["name"]:
            text += comment["selftext"]
            count += 1
            if count == 5:
                break
    r_text_list.append(text)
    id_list.append(post["name"])

In [47]:
len(r_text_list)

163

In [5]:
b_posts_lst = []
with open('bluesky_posts_0131.json', 'r') as file:
    for line in file:
        b_posts_lst.append(json.loads(line))


In [6]:
b_posts_lst 

[{'author': 'worldbytenews.bsky.social',
  'text': 'Renowned Dallas burger spot hosting fundraiser for California fire relief\u200bon January 25, 2025 at 3:44\xa0pm\n\nLauded Dallas chefs will sling burgers at the event this weekend with live music and proceeds going to the Pasadena Education Foundation. \u200bLauded Dallas chefs will sling burgers at the event…',
  'timestamp': '2025-01-31T12:31:07Z',
  'likes': 0,
  'reposts': 0,
  'comment': []},
 {'author': 'worldbytenews.bsky.social',
  'text': 'Renowned Dallas burger spot hosting fundraiser for California fire relief\u200bon January 25, 2025 at 3:44\xa0pm\n\nLauded Dallas chefs will sling burgers at the event this weekend with live music and proceeds going to the Pasadena Education Foundation. \u200bLauded Dallas chefs will sling burgers at the event…',
  'timestamp': '2025-01-31T12:20:41Z',
  'likes': 0,
  'reposts': 0,
  'comment': []},
 {'author': 'smcclain2110.bsky.social',
  'text': "I feel like when we were a more cohesive 

In [97]:
b_text_list = []

for post in b_posts_lst:
    text = post["text"]
    b_text_list.append(text)
    id_list.append(post["author"])

In [44]:
len(b_text_list)

4896

In [48]:
total_text = r_text_list + b_text_list

In [49]:
len(total_text)

5059

In [93]:
id_list =[]
order = 0
for i in total_text:
    id_list.append(order)
    order += 1


In [105]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# we add this to remove stopwords

my_stopwords = ["california", "fires", "fire","cafire","los","angeles","los angeles"]  # 将 'word1', 'word2', 'word3' 替换为你想去除的停用词

# add self-defined stopwords to english dictionary
combined_stopwords = list(ENGLISH_STOP_WORDS.union(set(my_stopwords)))
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=combined_stopwords)

model = BERTopic(
    vectorizer_model=vectorizer_model,
    language='english', calculate_probabilities=True,
    verbose=True
)
topics, probs = model.fit_transform(total_text)

Batches:   0%|          | 0/159 [00:00<?, ?it/s]

2025-03-04 22:00:14,852 - BERTopic - Transformed documents to Embeddings
2025-03-04 22:00:27,964 - BERTopic - Reduced dimensionality
2025-03-04 22:00:29,432 - BERTopic - Clustered reduced embeddings


In [106]:
model.get_topic_info()  # display topics


,Topic,Count,Name,Representation,Representative_Docs
0,-1,1672,-1_la_people_like_wildfires,"[la, people, like, wildfires, just, victims, t...",[An unapologetic takedown of Khloé Kardashian’...
1,0,303,0_battery_plant_storage_largest,"[battery, plant, storage, largest, worlds larg...",[Massive fire erupts at one of the world's lar...
2,1,281,1_southern_rain_winds_weather,"[southern, rain, winds, weather, risk, extreme...",[Fire risk and strong winds continue in Southe...
3,2,156,2_dallas_burgers event_sling burgers_chefs sling,"[dallas, burgers event, sling burgers, chefs s...",[Renowned Dallas burger spot hosting fundraise...
4,3,152,3_water_trump_water policy_policy,"[water, trump, water policy, policy, prepares,...",[Trump targets California water policy as he p...
...,...,...,...,...,...
73,72,11,72_air_califrnia_pilots_fighting,"[air, califrnia, pilots, fighting, cockpit vie...",[- Hughes Fire: California Air National Guard ...
74,73,11,73_post event_event attributes_attributes_http...,"[post event, event attributes, attributes, htt...",[\n\nhttps://www.nbclosangeles.com/news/califo...
75,74,10,74_devil winds_destruction malibu_winds httpsy...,"[devil winds, destruction malibu, winds httpsy...",[#california #Wildfire \n\nLIVE | Los Angeles ...
76,75,10,75_app_genasys_genasys protect_watch duty,"[app, genasys, genasys protect, watch duty, fr...",[App provides immediate fire information to Lo...


In [107]:
model.reduce_topics(total_text, nr_topics=15) 

2025-03-04 22:00:39,985 - BERTopic - Reduced number of topics from 78 to 15


In [108]:
freq = model.get_topic_info()
freq.head(15)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1672,-1_la_people_wildfires_like,"[la, people, wildfires, like, just, trump, vic...",[Another fire. 😢https://www.nbclosangeles.com/...
1,0,2685,0_trump_southern_battery_new,"[trump, southern, battery, new, water, wildfir...",[Fire at one of the world's largest battery pl...
2,1,170,1_relief_concert_benefit_fireaid,"[relief, concert, benefit, fireaid, help, dona...",[Billie Joe Armstrong of Green Day attends the...
3,2,156,2_dallas_burgers event_sling burgers_chefs sling,"[dallas, burgers event, sling burgers, chefs s...",[Renowned Dallas burger spot hosting fundraise...
4,3,135,3_home_home leaving_struggling stability_incre...,"[home, home leaving, struggling stability, inc...",[Dear Friends\nI hope this message finds you w...
5,4,63,4_eaton_edison_southern edison_deadly,"[eaton, edison, southern edison, deadly, deadl...",[California utility reports fault on power lin...
6,5,27,5_3891314_downey live_dispatch_downey,"[3891314, downey live, dispatch, downey, la do...","[⬆️\n\n#LosAngelesCounty, #California #Sheriff..."
7,6,24,6_additional details_0000_outlook_outlook posted,"[additional details, 0000, outlook, outlook po...","[SPC Day 1 Fire Weather Outlook posted at Wed,..."
8,7,24,7_post_politicslos_post event_wildfireseaton f...,"[post, politicslos, post event, wildfireseaton...",[\n\nhttps://www.nbclosangeles.com/news/califo...
9,8,23,8_tiktok_people_just_social,"[tiktok, people, just, social, tiktoks, reddit...",[Have you guys heard about the wildfires in LA...


In [109]:
import pandas as pd

# get the result after topic number reduced
topics, probs = model.transform(total_text)


probs_df = pd.DataFrame(probs, columns=[f"Topic_{i}" for i in range(probs.shape[1])])


assigned_topics = topics




Batches:   0%|          | 0/159 [00:00<?, ?it/s]

2025-03-04 22:02:03,688 - BERTopic - Reduced dimensionality
2025-03-04 22:02:05,898 - BERTopic - Calculated probabilities with HDBSCAN
2025-03-04 22:02:05,899 - BERTopic - Predicted clusters


In [112]:
  
topics_df = pd.DataFrame({
    'Document_ID': id_list,  
    'Assigned_Topic': assigned_topics,  
    'Topic_Probabilities': [probs[i][assigned_topics[i]] for i in range(len(topics))]  
})

# combine topic probability into df
topics_df = pd.concat([topics_df, probs_df], axis=1)


print(topics_df.head(15))

   Document_ID  Assigned_Topic  Topic_Probabilities       Topic_0  \
0   t3_1iazjin              -1         0.000000e+00  6.620513e-02   
1   t3_1i0nukq              -1         5.161709e-07  2.659071e-01   
2   t3_1i0788c               0         9.491622e-01  9.491622e-01   
3   t3_1i16xr6              -1         6.963381e-09  1.844093e-02   
4   t3_1hwsk85               0         3.978344e-01  3.978344e-01   
5   t3_1i0kavt              -1         8.317594e-11  3.931350e-08   
6   t3_1hxstm9              -1         1.534560e-15  5.006913e-02   
7   t3_1hy9y05               1         1.000000e+00  0.000000e+00   
8   t3_1i3cse3              -1         0.000000e+00  1.401694e-01   
9   t3_1hyw7g6               0         1.000000e+00  1.000000e+00   
10  t3_1hz6ake               9         1.000000e+00  0.000000e+00   
11  t3_1hx8wqc               0         4.592126e-01  4.592126e-01   
12  t3_1hyaku9               0         7.818725e-01  7.818725e-01   
13  t3_1i27dic              -1    

In [113]:
model.visualize_barchart()

In [100]:
topics_df.to_csv('topic_class.csv')

In [114]:
fig = model.visualize_barchart(topics=list(range(14)))
fig.show()
